### Global measures distributions in snapshots, displayed for each snapshot and joint for treatments

In [1]:
import os
import toml
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src import settings
from src.utils import fileio

CONFIG_PATH = os.path.join(settings.CONFIG_DIR, "main.toml")
with open(CONFIG_PATH, "r") as file:
    config = toml.load(file)

TIME_WINDOW = config["TIME_WINDOW"]
INPUT_PATH = os.path.join(settings.RESULTS_DIR, "global_measures_snapshots", f"{TIME_WINDOW}_sec_window")
all_treatments = fileio.load_multiple_folders(INPUT_PATH)
all_treatments = {key: value for key, value in all_treatments.items() if key in config["TREATMENTS"]}

dataframes = []
for treatment_name, treatment_path in all_treatments.items():
    all_groups = fileio.load_files_from_folder(treatment_path)
    treatment_dataframes = []
    for group_name, group_path in all_groups.items():
        df = pd.read_csv(group_path, index_col=0)
        df["Group"] = group_name
        treatment_dataframes.append(df)

    combined_df = pd.concat(treatment_dataframes)
    combined_df = combined_df.set_index(["Group"], append=True)
    combined_df.index.names = ["Snapshot", "Group"]
    combined_df = combined_df.reorder_levels(["Group", "Snapshot"])
    combined_df["Treatment"] = treatment_name
    combined_df = combined_df.set_index("Treatment", append=True)
    dataframes.append(combined_df)

combined_data = pd.concat(dataframes)

combined_data_reset = combined_data.reset_index()
for measure_name in combined_data.columns:

    plt.figure(figsize=(25, 3))
    sns.boxplot(data=combined_data_reset, x="Snapshot", y=measure_name, hue="Treatment")
    plt.xlabel("Snapshot")
    plt.ylabel(measure_name)
    plt.title(f"Distribution of {measure_name}")
    plt.legend(bbox_to_anchor=(1.0, 0.7), loc="upper left")
    plt.show()

SystemExit: /srv/milky/drosophila-datasets/drosophila-isolation/data/results/global_measures_snapshots/60_sec_window is invalid path!

/home/milky/drosophila-isolation/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
